In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = '/datasets/book-rec/'

BOOK_RATINGS_FILE = DATA_PATH + 'BX-Book-Ratings.csv'
BOOKS_FILE = DATA_PATH + 'BX-Books.csv'
USERS_FILE = DATA_PATH + 'BX-Users.csv'


In [6]:
df_ratings = pd.read_csv(BOOK_RATINGS_FILE, delimiter=';',encoding='latin-1', error_bad_lines=False)
df_ratings = df_ratings.rename(columns={'User-ID':'userId', 'ISBN':'itemId', 'Book-Rating':'rating'})
df_ratings.head()

,userId,itemId,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
df_books = pd.read_csv(BOOKS_FILE, delimiter=';',encoding='latin-1', error_bad_lines=False)
df_books.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:30

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
df_users = pd.read_csv(USERS_FILE, delimiter=';',encoding='latin-1', error_bad_lines=False)
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
len(df_ratings['User-ID'].unique()), len(df_ratings['ISBN'].unique())
df_ratings['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2])

In [7]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate

reader = Reader(line_format='user item rating', rating_scale=(1, 10))
dataset = Dataset.load_from_df(df_ratings, reader)


algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True, n_jobs=8)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.5031  3.4920  3.4999  3.4966  3.5003  3.4984  0.0038  
MAE (testset)     2.9302  2.9228  2.9292  2.9249  2.9309  2.9276  0.0032  
Fit time          47.89   48.15   48.13   47.78   47.56   47.90   0.22    
Test time         2.04    2.03    2.07    1.97    2.03    2.03    0.03    

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

user_enc = LabelEncoder()
df_ratings['user'] = user_enc.fit_transform(df_ratings['userId'].values)
n_users = df_ratings['user'].nunique()

item_enc = LabelEncoder()
df_ratings['item'] = item_enc.fit_transform(df_ratings['itemId'].values)
n_movies = df_ratings['item'].nunique()
df_ratings['rating'] = df_ratings['rating'].values.astype(np.float32)

X = df_ratings[['user', 'item']].values
y = df_ratings['rating'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [70]:
import tensorflow as tf
from tensorflow import keras
 

In [32]:
# from keras.layers import Input, Reshape, Dot
# from keras.layers.embeddings import Embedding
# from keras.optimizers import Adam
# from keras.regularizers import l2
# def RecommenderV1(n_users, n_movies, n_factors):
#     user = Input(shape=(1,))
#     u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
#                   embeddings_regularizer=l2(1e-6))(user)
#     u = Reshape((n_factors,))(u)
#     
#     movie = Input(shape=(1,))
#     m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
#                   embeddings_regularizer=l2(1e-6))(movie)
#     m = Reshape((n_factors,))(m)
#     
#     x = Dot(axes=1)([u, m])
#     model = Model(inputs=[user, movie], outputs=x)
#     opt = Adam(lr=0.001)
#     model.compile(loss='mean_squared_error', optimizer=opt)
#     return model

ModuleNotFoundError: No module named 'keras'